# import libaries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# define class for solow model

In [2]:
class solow:
    """
    code for computing and plotting the solow growth model
    """
    def __init__(me, η=0.01,  # Population growth rate
                     γ=0.02,  # Technology growth rate
                     s=0.20,  # Savings rate
                     δ=0.10,  # Capital depreciation rate
                     θ=0.33,  # Capital share of production
                     A0=1.0,  # Initial productivity
                     K0=1.0,  # Initial capital stock
                     N0=1.0,  # Initial population
                     T=81,    # Number of periods to solve for
                     path='./Figures/'):  # File path to save figures 
        
        # set class inputs to be class parameters
        me.η, me.γ, me.s, me.δ, me.θ = η, γ, s, δ, θ
        me.A0, me.K0, me.N0, me.T = A0, K0, N0, T
        me.path = path
        me.g = (1 + me.γ) * (1 + me.η) - 1
    
    
    def solve_t(me, k, t):     
        """
        solves period t of the solow model given capital per effective worker, k.
        ----------
        PARAMETERS
        ----------
        k : float
            current capital per effective worker.
        t : int
            time period.
        -------
        RETURNS
        -------
        {flt x 5}, flt
            wage, rental rate, interest rate, output, consumption, ...
            ...and next period capital, all per effective worker.
        """
        
        # initialize period solutions
        v_t = {}
    
        v_t['w'] = (1 - me.θ) * k**me.θ   # note: this is the wage per effective unit of labor. The wage per person is A_t * w.
        v_t['R'] = me.θ * k**(me.θ - 1)
        v_t['r'] = (1 + v_t['R']) * (1 - me.δ) - 1
        
        v_t['y'] = k**me.θ
        v_t['c'] = (1 - me.s) * v_t['y']
        kk = (me.s * v_t['y'] + (1 - me.δ) * k) / (1 + me.g)
        
        return [v_t, kk]
    
    
    
    def solve_path(me):
        """
        solves first me.T periods of the solow model.
        ----------
        PARAMETERS
        ----------
        None.
        -------
        RETURNS
        -------
        [np.array(float: me.Tx1) x 6]
            wage, rental rate, interest rate, output, consumption, ...
            ...and capital, all per effective worker.
        """        
        
        # initialize equilibrium paths
        v = {i: np.zeros(me.T + 1) for i in ['w', 'R', 'r', 'y', 'c', 'k']} 
        
        # solve for model variables
        v['k'][0] = me.K0 / (me.A0 * me.N0)
        for t in range(me.T):
            
            # solve period outcomes
            v_t, v['k'][t+1] = me.solve_t(v['k'][t], t)
            
            # add period solutions to path 
            for key, val in v_t.items():

                v[key][t] = val
            
        return v
        
        
        
    def solve_ss(me):
        """
        solves for steady state of the solow model.
        ----------
        PARAMETERS
        ----------
        None.
        -------
        RETURNS
        -------
        {flt x 6}
            steady state wage, rental rate, interest rate, output, ...
            ...consumption, and next period capital, ...
            ...all per effective worker.
        """         
        
        # initialize steady state solution
        v_ss = {}        
               
        # solve for steady state variables
        v_ss['k'] = (s / ((1 + me.g) - (1 - me.δ)))**(1 / (1 - me.θ))
        
        v_ss['w'] = (1 - me.θ) * v_ss['k']**me.θ               # Note: this is the wage per effective unit of labor. The wage per person is A_t * w.
        v_ss['R'] = me.θ * v_ss['k']**(me.θ - 1)
        v_ss['r'] = (1 + v_ss['R']) * (1 - me.δ) - 1
        
        v_ss['y'] = v_ss['k']**me.θ
        v_ss['c'] = (1 - me.s) * v_ss['y']
        
        return v_ss
                  

# define plotting functions

In [3]:
####################################
#
#
#   define functions to plot outcomes
#
#
#####################################

def plot_variable(lbl, var1, var2, var1ss, var2ss):
    """
    plots two sets of model solutions for a given variable
    ----------
    parameters
    ----------
    lbl     : str
            variable label
    var1    : np.array(T)
            variable path given parameters 1
    var2    : np.array(T)
            variable path given parameters 2  
    var1ss  : flt
            variable ss value given parameters 1
    var2ss  : flt
            variable ss value given parameters 2         
    -------
    returns
    -------
    None.
    """
    
    # initialize figure
    fig, ax = plt.subplots()
    
    # plot paths
    ax.plot(var1, marker='o', markevery=5, label='p1', color='tab:blue')
    ax.plot(var2, marker='^', markevery=5, label='p2', color='tab:orange')
    
    # plot steady state values
    ax.axhline(y=var1ss, ls='--', lw=0.5, color='tab:blue')
    ax.axhline(y=var2ss, ls='--', lw=0.5, color='tab:orange')
    
    # finish figure
    ax.set_ylim(bottom=0)
    ax.set_xlabel('Time, T')
    ax.set_ylabel(lbl)
    ax.legend(frameon=False)
    plt.show()
    
    

def plot_eq(solow1, solow2):
    """
    plots model solutions for all variables given two parameter sets p1 and p2
    ----------
    parameters
    ----------
    solow1  : class
            instance of the class "solow"
    solow2  : class
            instance of the class "solow" with alternative set of parameters        
    -------
    returns
    -------
    {np.array(float: T x 1) x 6}, 
    {np.array(float: T x 1) x 6}, 
    {flt x 6}, {flt x 6}
        wage, rental rate, interest rate, output, consumption, ...
        ...and capital, all per effective worker.
        equilibrium path, then steady state values ...
        alternately for parameter set 1 and 2.
    """
    
    # solve for paths
    v1 = solow1.solve_path()
    v2 = solow2.solve_path()
    
    # solve for steady states
    v1_ss = solow1.solve_ss()
    v2_ss = solow2.solve_ss()
    
    # iterate through all variables
    for i in v1.keys():
        
        # plot outcomes for variable
        plot_variable(i, v1[i][:-1], v2[i][:-1], v1_ss[i], v2_ss[i])
    
    return v1, v2, v1_ss, v2_ss

# set parameters and solve for equilibrium

In [ ]:
####################################
#
#
#   set parameters
#
#
#####################################

# model parameters: I
solow1 = solow()

# model parameters: II
solow2 = solow(s=0.1)



####################################
#
#
#   solve for equilibria and plot
#
#
#####################################

v1, v2, v1_ss, v2_ss = plot_eq(solow1, solow2)